# 6장. Profile Page and Avatars

- 사용자 정보가 표시되는 페이지 작성

#### User Profile Page

- 사용자의 프로필이 보여질 페이지를 작성
- user.html을 통해 페이지를 선언 
- 페이지를 만들기만 해선 브라우저가 존재하지 않음 즉, 링크해주는 과정이 필요함
- base.html에 아래와 같은 문장을 추가하여 링크함

```html
<a href="{{ url_for('user', username=current_user.username) }}">Profile</a>
```

- http://localhost:5000/user/username 과 같은 형식으로 사용자 프로필 페이지가 생성

#### Avatars

- 프로필 페이지에 사진을 추가
- Gravatar 서비스 사용
- Gravatar URL로 접속시 Gravatar가 나옴

##### Gravatar URL 얻는 방법

```python
from hashlib import md5
'https://www.gravatar.com/avatar/' + md5(b'john@example.com').hexdigest()
'https://www.gravatar.com/avatar/d4c74594d841139328695756648b6bd6' //return URL
```

- Avatars는 사용자 프로필과 연결되어 있기에 Gravatar URL을 생성하는 로직을 사용자 모델에 추가
- 아래 함수를 models.py 모듈에 추가

```python
def avatar(self, size):
    digest = md5(self.email.lower().encode('utf-8')).hexdigest()
    return 'https://www.gravatar.com/avatar/{}?d=identicon&s={}'.format(
        digest, size)
```

##### 섹션을 나눠 여러 개의 아바타 넣기


```html
<table>
    <tr valign="top">
        <td><img src="{{ user.avatar(128) }}"></td>
        <td><h1>User: {{ user.username }}</h1></td>
    </tr>
</table>
```

- 위 코드와 같이 섹션을 나누어 아바타 추가
- 위 코드를 작성한 만큼 섹션이 나누어짐

#### Using Jinja2 Sub-Templates

- sub-templates를 작성하여 변경하고자 하는 부분만 업데이트가 되도록 구현
- template 폴더에 sub-templates 파일을 작성한 후, base.html에 링크


ex) _post.html
```html
    <table>
        <tr valign="top">
            <td><img src="{{ post.author.avatar(36) }}"></td>
            <td>{{ post.author.username }} says:<br>{{ post.body }}</td>
        </tr>
    </table>
```

#### More Interesting Profiles

- 프로필에 사용자가 사이트에 마지막으로 접속한 시간을 표시하는 등 더 많은 정보를 표시할 수 있음


##### Migration

- 데이터베이스가 수정될 때마다 데이터베이스 마이그레이션을 생성
- 데이터를 파괴하지 않고, Migration script의 변경 사항을 적용할 수 있음

```shell
$ flask db migrate -m "new fields in user model"
```

#### Recording The Last Visit Time For a User

- 사용자가 서버에 요청할 때마다 시간을 표시해주는 기능

```python
@app.before_request
def before_request():
    if current_user.is_authenticated:
        current_user.last_seen = datetime.utcnow()
        db.session.commit()
```

# Profile Editor

- 사용자가 자신의 정보를 입력할 수 있는 기능

```python
def edit_profile():
    form = EditProfileForm()
    if form.validate_on_submit():
        current_user.username = form.username.data
        current_user.about_me = form.about_me.data
        db.session.commit()
        flash('Your changes have been saved.')
        return redirect(url_for('edit_profile'))
    elif request.method == 'GET':
        form.username.data = current_user.username
        form.about_me.data = current_user.about_me
    return render_template('edit_profile.html', title='Edit Profile',
                           form=form)
```

- validate_on_submit()이 True를 반환하면 데이터베이스에 작성
- validate_on_submit()이 False면 2가지 이유가 존재
    1. 브라우저가 GET 요청을 보낸 경우
    2. 브라우저가 데이터와 함께 POST 요청을 보냈지만 데이터가 잘못된 경우